In [ ]:
# Example: Fetching connectivity SMILES from PubChemPy (no deprecation warning)
import pubchempy as pcp

compounds = pcp.get_compounds('aspirin', 'name')
for c in compounds:
    print(f"CID: {c.cid}, Connectivity SMILES: {c.connectivity_smiles}")

# Chapter 2: Target Identification - Interactive Notebook

This notebook demonstrates code and concepts from [Chapter 2: Target Identification](../chapters/chapter2-target-identification.qmd) of the book.

You can run and modify the code cells below to explore AI-powered target mining hands-on.

## Check for Existing Notebooks

Let's search the current directory for Jupyter notebook files.

In [1]:
import os

# List all .ipynb files in the current directory
notebooks = [f for f in os.listdir('.') if f.endswith('.ipynb')]
print("Notebooks in this directory:", notebooks)

Notebooks in this directory: ['chapter3-cheminformatics-essentials.ipynb', 'chapter1-genomics-to-molecules.ipynb', 'chapter4-compound-screening.ipynb', 'chapter5-ml-for-drug-discovery.ipynb', 'chapter2-target-identification.ipynb']


## Generate a New Notebook File

You can programmatically create a new Jupyter notebook using the `nbformat` library.

In [2]:
import nbformat as nbf

nb = nbf.v4.new_notebook()
nb['cells'] = [nbf.v4.new_code_cell("print('Hello, new notebook!')")]
with open('sample_generated_notebook.ipynb', 'w') as f:
    nbf.write(nb, f)
print("Created sample_generated_notebook.ipynb")

Created sample_generated_notebook.ipynb


## List All Notebooks in the Directory

Let's display all `.ipynb` files in the current directory again.

In [3]:
# List all .ipynb files again
notebooks = [f for f in os.listdir('.') if f.endswith('.ipynb')]
print("Notebooks in this directory:", notebooks)

Notebooks in this directory: ['chapter3-cheminformatics-essentials.ipynb', 'sample_generated_notebook.ipynb', 'chapter1-genomics-to-molecules.ipynb', 'chapter4-compound-screening.ipynb', 'chapter5-ml-for-drug-discovery.ipynb', 'chapter2-target-identification.ipynb']


## Open a Notebook in Visual Studio Code

You can open a notebook from the terminal using:

```bash
code chapter2-target-identification.ipynb
```

## Run Code Cells Programmatically

You can execute notebook cells programmatically using `nbconvert` or `papermill`. Example with papermill:

```python
import papermill as pm
pm.execute_notebook('chapter2-target-identification.ipynb', 'output.ipynb')
```
